# 使用 Tool 

1. 安装对应 Tool
2. 确定要执行的 Tool
3. 执行 Tool
4. 整合后的结果

In [5]:
expression_evaluator_tool = {
        'type': 'function',
        'function': {
          'name': 'expression_evaluator',
          'description': 'Evaluates a mathematical expression following the PEMDAS/BODMAS order of operations.',
          'parameters': {
            'type': 'object',
            'properties': {
              'expression': {
                'type': 'string',
                'description': 'The mathematical expression to evaluate. The expression can include integers, decimals, parentheses, and the operators +, -, *, and /.',
              }
            },
            'required': ['expression'],
          },
        },
      }

import numexpr as ne

def expression_evaluator(expression = ""):
    try:
        result = ne.evaluate(expression)
        return f"Answer to {expression} is {result}"
    except Exception as e:
        return str(e)

In [15]:
!export OLLAMA_HOST="localhost:6060"

In [16]:
import ollama
tool_history = []

def tool_calling(query):
    
    messages = [{'role': 'user', 'content': query}]

    # response = ollama.chat(
    response = ollama.chat(    
        model='llama3.1:latest',
        messages=messages,
        tools=[expression_evaluator_tool,],
    )

    print(response['message'])
    
    messages.append(response['message'])

    if not response['message'].get('tool_calls'):
        print("The model didn't use the function. Its response was:")
        print(response['message']['content'])

    if response['message'].get('tool_calls'):
        available_functions = {
            'expression_evaluator' : expression_evaluator
            # 'wikipedia' : wikipedia,
            # 'weather_forecast' : weather_forecast
        }
    for tool in response['message']['tool_calls']:

        function_to_call = available_functions[tool['function']['name']]
        args = tool['function']['arguments'].values()
        function_response = function_to_call(*args)

        print("\nTool Response:\n", function_response)

        messages.append({'role': 'tool', 'content': function_response})

    final_response = ollama.chat(model='llama3.1', messages=messages)

    return final_response['message']['content']

In [19]:
query = '请计算 (100+2)**3  用中文解答过程'
result = tool_calling(query)
print("\n\nAnswer:\n", result)

{'role': 'assistant', 'tool_calls': [{'function': {'name': 'expression_evaluator', 'arguments': {'expression': '(100+2)**3'}}}]}

Tool Response:
 Answer to (100+2)**3 is 1061208


Answer:
 为了计算 (100+2)**3，我们首先要计算表达式内的内容，也就是 100 和 2 的和。然后将结果提升到 3 次方。

1. 计算表达式内的内容：100 + 2 = 102
2. 将 102 提升到 3 次方：102 ** 3 = 1061208

因此，(100+2)**3 等于 1061208。
